# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], 
                          pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], 
                          pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', 
                                             ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,29.316592,0.376962,0.435539,0.029679,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.976661,0.973944,0.968553,0.974843,0.974491,0.003350,1
11,48.796049,0.599538,0.373767,0.085011,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.976661,0.974843,0.968553,0.971249,0.973593,0.003202,2
8,56.895895,0.288117,0.799907,0.285697,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.979354,0.975763,0.973944,0.966757,0.970350,0.973234,0.004350,3
6,4.865381,0.604541,0.224437,0.027809,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.981149,0.976661,0.973046,0.961366,0.973944,0.973233,0.006569,4
10,32.134929,1.079514,0.420857,0.199270,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975763,0.975763,0.973944,0.969452,0.970350,0.973055,0.002674,5


In [5]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,51.418810,0.961167,0.359782,0.056243,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978456,0.973070,0.974843,0.968553,0.970350,0.973055,0.003463,1
8,50.459166,0.276335,0.890692,0.354304,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.973070,0.974843,0.967655,0.970350,0.972695,0.003444,2
10,29.149610,1.167020,0.538820,0.337914,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976661,0.974865,0.972147,0.970350,0.969452,0.972695,0.002711,3
7,28.945865,0.795124,0.396452,0.039517,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976661,0.972172,0.972147,0.966757,0.972147,0.971977,0.003140,4
5,46.237157,0.327683,0.545836,0.041078,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.974865,0.970377,0.971249,0.964061,0.969452,0.970001,0.003491,5
